<a href="https://colab.research.google.com/github/OscarRojasG/Experimentos-GPTValidator/blob/main/Framework_Experimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Framework para experimentos GPTValidator

La estructura de carpetas y archivos es la siguiente:

* Datasets
* Miniprompts
  * Ejemplos
  * Contexto
  * Pregunta
  * Respuesta
  * Criteria
  * Reflection
  * Feedback
  * Score
  * Output
* Results

La idea es que los resultados de los experimentos sean guardados en archivos después de cada ejecución.

Para cada experimento se guardarán los siguientes datos:
* Nombre del dataset
* Fecha de ejecución
* Datos por iteración:
  * Puntajes asignados
  * Métricas obtenidas (MSE, RMSE, R^2)
* Resumen de métricas (Media, Desviación estándar)

# Framework para experimentos

In [1]:
!pip install openai==0.28 &> /dev/null
!pip install openai-multi-client &> /dev/null
!git clone https://github.com/rilianx/GPTEvaluator &> /dev/null

In [2]:
!unzip -n Experiments.zip &> /dev/null

In [3]:
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report, r2_score, accuracy_score
from GPTEvaluator.GPTEvaluator import chat_gpt_multiple
from openai_multi_client import OpenAIMultiClient
from datetime import datetime, timedelta
from google.colab import userdata
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import openai
import getpass
import json
import re

openai.api_key = userdata.get('api_key')

# Genera un prompt a partir de los miniprompts especificados en el diccionario data
def generate_prompt(data):
  def read_file(folder, filename):
    path = f"Experiments/Miniprompts/{folder}/{filename}"
    try:
      return open(path, 'r', encoding='utf-8').read()
    except:
      raise Exception(f"Error: El archivo {path} no existe")

  prompt = ""
  n = len(data.items())

  for i, (key, value) in enumerate(data.items()):
    if key == "instructions":
      prompt += "Instructions: \n"
      for j, (key2, value2) in enumerate(data[key].items()):
        miniprompt = read_file(key2, value2)
        prompt += miniprompt + "\n\n"
    else:
      miniprompt = read_file(key, value)
      prompt += miniprompt

      if i < n-1:
        prompt += "\n\n"

  return prompt

# Carga un dataset a partir de un archivo xlsx y valida sus columnas
def load_dataset(filename, column_data):
  path = f"Experiments/Datasets/{filename}"
  df = pd.read_excel(path)

  mandatory_cols = ["context", "question", "answer", "real_eval"]
  for key in mandatory_cols:
    if key not in column_data.keys():
      raise Exception(f"Error: Debe especificar la columna para la variable {key}")

    value = column_data[key]
    if value not in df.columns:
      raise Exception(f"Error: La columna {value} no existe")

    df = df.rename(columns={value: key})

  return df

# Genera las respuestas con ChatGPT
def eval_gpt(df, prompt):
  api = OpenAIMultiClient(endpoint="chats", data_template={"model": "gpt-3.5-turbo", "temperature": 0.2, "n": 1, "timeout":10}, concurrency=50, wait_interval=1, max_retries=3, retry_max=10, retry_multiplier=1)

  texts = []
  for i, row in df.iterrows():
    text = prompt.format(Question=row['question'], Answer=row['answer'], Context=row['context'])
    texts.append(text)

  answers_gpt = chat_gpt_multiple(api, texts)
  return answers_gpt

# Calcula los puntajes obtenidos por GPT
def get_gpt_scores(answers_gpt, score_function):
  # Convierte la respuesta de GPT en un diccionario
  def get_gpt_dict(answer_gpt):
    pattern = r'\{[^{}]+\}'
    answer = re.findall(pattern, answer_gpt)[0]
    return eval(answer)

  gpt_scores = []
  for answer in answers_gpt:
    try:
      gpt_dict = get_gpt_dict(answer[0])
    except:
      print(f"Error al extraer diccionario. Respuesta GPT: \n{answer[0]}\n\n")
      gpt_scores.append(None)
      continue

    try:
      score = score_function(gpt_dict)
      gpt_scores.append(score)
    except:
      print(f"Error al calcular puntaje. Respuesta GPT: \n{answer[0]}\n\n")
      gpt_scores.append(None)

  return gpt_scores

# Obtiene los puntajes reales de un dataset
def get_real_scores(df):
    return df['real_eval'].tolist()

# Calcula las métricas de evaluación
def get_stats(real_scores, gpt_scores):
  sample_size = len(real_scores)
  for i in reversed(range(sample_size)):
    if gpt_scores[i] == None:
      real_scores.pop(i)
      gpt_scores.pop(i)

  stats = {
      "confusion_matrix": confusion_matrix(real_scores, gpt_scores).tolist(),
      "mse": mean_squared_error(real_scores, gpt_scores),
      "mae": mean_absolute_error(real_scores, gpt_scores),
      "r2": r2_score(real_scores, gpt_scores),
      "accuracy": accuracy_score(real_scores, gpt_scores),
      "sample_size": len(real_scores)
  }
  return stats

def save_results(dataset, prompt, stats, dataset_size, filepath):
  with open(filepath, 'w', encoding='utf-8') as file:
    data = {
        "dataset_name": dataset,
        "dataset_size": dataset_size,
        "prompt": prompt,
        "stats": stats
    }
    json.dump(data, file, ensure_ascii=False)

# Evalúa un prompt y retorna las estadísticas obtenidas
def evaluate_prompt(df, prompt, score_function):
  try:
    answers_gpt = eval_gpt(df, prompt)
    real_scores = get_real_scores(df)
    gpt_scores = get_gpt_scores(answers_gpt, score_function)
    return get_stats(real_scores, gpt_scores)
  except Exception as e:
    print(e)
    return

# Evalúa un prompt para N repeticiones
def evaluate_prompt_n(dataset, column_data, prompt_data, score_function, repetitions):
  df = load_dataset(dataset, column_data)
  prompt = generate_prompt(prompt_data)
  dataset_size = len(df)
  stats_list = []

  for _ in range(repetitions):
    stats = evaluate_prompt(df, prompt, score_function)
    stats_list.append(stats)

  date = datetime.now() - timedelta(hours=4)
  formatted_date = date.strftime('%Y%m%d-%H%M')

  filepath = f"Experiments/Results/{formatted_date}.json"
  save_results(dataset, prompt, stats_list, dataset_size, filepath)

# Retorna una lista con la metadata necesaria para generar cada prompt
def get_prompt_data_list(prompt_data):



# Evalúa varios prompts a la vez con N repeticiones
def experiment(dataset, column_data, prompt_data, score_function, repetitions):
  prompt_data_list = get_prompt_data_list()
  print(prompt_data_list)

In [4]:
prompt_data = {
    "examples": "examples_1.txt",
    "context": "context_1.txt",
    "question": "question_1.txt",
    "answer": "answer_1.txt",
    "instructions": {
        "reflection": "reflection_1.txt",
        "feedback": "feedback_1.txt",
        "score": "score_1.txt",
    },
    "criteria": "criteria_1.txt",
    "output": "output_1.txt"
}

column_data = {
    "context": "Contexto",
    "question": "Pregunta",
    "answer": "Respuesta",
    "real_eval": "EvalProfe"
}

def score_function(gpt_dict):
  score = 0.5 * gpt_dict['correctness'] + 0.3 * gpt_dict['completeness'] + 0.2 * gpt_dict['clarity']
  if score <= 2: return 0
  elif score <= 5: return 1
  elif score <= 8: return 2
  return 3

evaluate_prompt_n("test.xlsx", column_data, prompt_data, score_function, 2)

2-1-0-2-0-1-

In [ ]:
print(generate_prompt(prompt_data))

**Examples:**
Q: ¿Cómo se podría implementar un historial de navegación web usando dos pilas? El historial debe permitir ir hacia atrás y adelante con las páginas previamente visitadas. Describa un algoritmo.
Incorrect Answer: Usamos dos pilas para ir hacia adelante y hacia atrás en el historial.  (Score: 0)

Q: ¿Cómo se busca un valor en un árbol rojo-negro? Explique el proceso paso a paso.
Incorrect Answer: PAra buscar el valor en un árbol rojo-negro debemos pasar por nodos rojos y negros hasta encontrar el valor. (Score: 0)

Q: ¿Por qué el acceso a un elemento específico en un arreglo es O(1), es decir, no depende de la cantidad de datos?
Incorrect Answer: El acceso es O(1) por que toma un tiempo constante y no depende de la cantidad de datos. (Score: 0)

Q: ¿Cuando se recomienda utilizar arreglos en vez de listas enlazadas? Haga referencia a complejidades temporales en su explicación.
Incorrect Answer: Un arreglo es recomendable en determinadas situaciones, mientras que la lista en